In [155]:
import numpy as np

In [156]:
class XavierInitializer:
    def __init__(self, n_nodes1):
        self.sigma = 1.0 / n_nodes1**(1/2)

    def W(self, n_nodes1, n_nodes2):
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        return self.sigma * np.random.randn(n_nodes2)

In [157]:
class SGD:
    """
    Stochastic gradient descent
    Parameters
    ----------
    lr : Learning rate
    """
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, layer):
        """
        Update weights and biases for a layer
        Parameters
        ----------
        layer : Instance of the layer before update
        """
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return layer

<h2>[Problem 1] Creating a one-dimensional convolutional layer class that limits the number of channels to one</h2>

In [158]:
class SimpleConv1d:
    def __init__(self, F, initializer, optimizer):
        self.F = F
        self.optimizer = optimizer
        self.W = initializer.W(1, F)[0]
        self.B = initializer.B(1)[0]

    def forward(self, X):
        self.X = X
        A = np.empty(compute_output_size(len(X), 0, self.F, 1))
        for i in np.arange(0, len(A)):
            A[i] = np.dot(X[i:i+self.F], self.W) + self.B
        return A

    def backward(self, dA):
        #update dW
        self.dW = np.empty(self.F)
        for s in np.arange(0, self.F):
            self.dW[s] = np.dot(dA, self.X[s:s+len(dA)])
        #update dB
        self.dB = np.sum(dA)
        #update dZ
        self.dZ = np.empty(len(self.X))
        for j in np.arange(0, len(self.X)):
            total = 0.0
            for s in np.arange(0, self.F):
                if (j - s >= 0 and j - s < len(dA)):
                    total += dA[j - s] * self.W[s]
            self.dZ[j] = total
        # update new W, B
        self = self.optimizer.update(self)
        return self.dZ

<h2>[Problem 2] Output size calculation after one-dimensional convolution</h2>

In [159]:
def compute_output_size(n_in, P, F, S):
    return int(1 + (n_in + 2*P - F) / S)

<h2>[Problem 3] Experiment of one-dimensional convolutional layer with small array</h2>

<h3>Forward</h3>

In [160]:
x = np.array([1,2,3,4]).astype(np.float64)
w = np.array([3, 5, 7]).astype(np.float64)
b = np.array([1]).astype(np.float64)

In [161]:
simplec1d = SimpleConv1d(len(w), XavierInitializer(len(x)), SGD())
simplec1d.B = b
simplec1d.W = w

In [162]:
A = simplec1d.forward(x)
print("A: {}".format(A))

A: [35. 50.]


<h3>Backward</h3>

In [163]:
dA = np.array([10, 20]).astype(np.float64)
simplec1d.backward(dA)
print("dW: {}\ndB: {}\ndZ: {}".format(simplec1d.dW, simplec1d.dB, simplec1d.dZ))

NameError: name 'dZ' is not defined

In [ ]:
class Scratch1dCNNClassifier:
    def __init__(self):
        pass